# PequniaAI: Personalized AI Agent Training
In this notebook, we will train personalized AI agents based on their behavior and specific traits. Each agent is described by attributes like age, gender, behavior, and unique traits. We'll process the data, train a neural network model, and evaluate the agents' responses to different input prompts.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv("data.csv")

data.head()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(df):
    tokens = tokenizer(df['Specific Traits'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=50)
    return tokens

tokens = preprocess_data(data)

label_encoder = LabelEncoder()
data['Behavior_Label'] = label_encoder.fit_transform(data['Behavior'])

X_train, X_test, y_train, y_test = train_test_split(tokens['input_ids'], data['Behavior_Label'], test_size=0.2, random_state=42)


In [ ]:
# Define the neural network model using an LSTM for sequential text processing
class TraitPredictionNN(nn.Module):
    def __init__(self, hidden_size=128):
        super(TraitPredictionNN, self).__init__()

        # Embedding layer and LSTM layer
        self.embedding = nn.Embedding(30522, hidden_size)  # BERT vocab size
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, 3)  # 3 possible behavior classes (friendly, professional, humorous)

    def forward(self, x):
        # Forward pass through the layers
        x = self.embedding(x)
        x, (hn, cn) = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take the last LSTM output
        x = self.output_layer(x)
        return x

# Initialize model, loss function, and optimizer
model = TraitPredictionNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Train the model on the data
def train_model(model, X_train, y_train, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass
        output = model(X_train)

        # Calculate loss
        loss = criterion(output, y_train)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Convert X_train and y_train to tensors
X_train_tensor = torch.tensor(X_train.tolist(), dtype=torch.long)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

# Train the model
train_model(model, X_train_tensor, y_train_tensor, epochs=5)


In [ ]:
# Function to evaluate the model's accuracy on the test set
def evaluate_model(model, X_test, y_test):
    model.eval()

    # Convert X_test and y_test to tensors
    X_test_tensor = torch.tensor(X_test.tolist(), dtype=torch.long)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    with torch.no_grad():
        output = model(X_test_tensor)
        _, predicted = torch.max(output, 1)
        accuracy = (predicted == y_test_tensor).float().mean()

    print(f'Accuracy: {accuracy:.4f}')

# Evaluate the model
evaluate_model(model, X_test, y_test)


In [ ]:
# Function to predict the behavior of a new input
def predict_behavior(model, input_text):
    model.eval()

    # Tokenize the input text
    input_tokens = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt", max_length=50)

    # Get the prediction from the model
    with torch.no_grad():
        output = model(input_tokens['input_ids'])
        _, predicted = torch.max(output, 1)

    # Decode the predicted label to its corresponding behavior
    predicted_behavior = label_encoder.inverse_transform(predicted.numpy())
    return predicted_behavior[0]

# Example prediction
test_input = "I like to tell jokes and make people laugh, but I always maintain professionalism."
predicted_behavior = predict_behavior(model, test_input)
print(f"Predicted Behavior: {predicted_behavior}")


In [ ]:
# Save the trained model to a file for future use
torch.save(model.state_dict(), "pequniaAI_model.pth")


In [ ]:
# Load the trained model from the saved file
model.load_state_dict(torch.load("pequniaAI_model.pth"))

# Example prediction using the loaded model
test_input = "I prefer concise and formal communication, focusing on technical topics."
predicted_behavior = predict_behavior(model, test_input)
print(f"Predicted Behavior: {predicted_behavior}")
